In [ ]:
#imports
!pip install wikipedia
!pip install fasttext
!pip install gensim
!pip install nltk

import string
import pandas as pd
import wikipedia
import fasttext
import fasttext.util
from pprint import pprint as print
from gensim.models.fasttext import FastText as fasttext
from gensim.test.utils import datapath
from google.colab import drive
import xml.etree.ElementTree as ET
import re
import nltk
from nltk.stem.snowball import SnowballStemmer
import fasttext.util
import sklearn as sk
from sklearn import svm
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import numpy as np

In [ ]:
#initial setup
nltk.download('punkt')
nltk.download('stopwords')

fasttext.util.download_model('it', if_exists='ignore') 
ft = fasttext.load_model('cc.it.300.bin')

wikipedia.set_lang("it")

drive.mount('/content/drive')


In [ ]:
#dataset init

#file consisting of pairs of target and prerequisite concepts (A, B) labelled as follows:
#1 if B is a prerequisite of A;
#0 in all other cases.
df = pd.read_csv('/content/drive/My Drive/train.csv')
train = df.sample(frac=0.8,random_state=200) 
validation = df.drop(train.index)

#The Wikipedia page of each concept found in the previous file.
#Each Wikipedia page is introduced by a `<doc>` element (with *id* and *url*) 
#containing the title and the text of the corresponding page.
tree = ET.parse('/content/drive/My Drive/dataset.xml')
pages = tree.getroot()

In [ ]:
#create dictionary of tokenized documents
punct = string.punctuation + '«``»' + "''"
it_stop_words = nltk.corpus.stopwords.words('italian')
stemmer = nltk.stem.snowball.ItalianStemmer(True)

doc_dict = {}

for content in pages.iter('doc'):
  #document to lowercase, replaced apostrophe with space since the tokenizer isn't able to split words like "l'addizione"
  document = (content.find('title').text  + content.find('text').text).replace("'"," ")
  #tokenized
  tokenized = nltk.tokenize.word_tokenize(document, "italian")
  #no punctuation and lowercase
  no_punct = [x.lower() for x in tokenized if x not in punct]
  #remove stop words
  no_stop_words = [x for x in no_punct if x not in it_stop_words]
  #stem
  #stemmed = [stemmer.stem(i) for i in no_stop_words]
  
  #add tokenized document
  doc_dict[content.find('title').text] = no_stop_words